In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from cosy.models import CosyNet
from cosy.losses import trace_norm, MaskedMeanSquaredError, mutual_information, l1_norm, wasserstein_distance
from cosy.callbacks import EarlyStoppingMultiLoss

In [2]:
df = pd.read_csv("/Users/tompope/Documents/python_enviroment/SoftwareDevelopment/soft_parameter_sharing/data/desulferization_data_w_tobacc.csv")
df.head()

,Unnamed: 0,framework,CO2_tp_lod,N2_tp_lod,SO2_tp_lod,RDF_electronegativity_2.00,RDF_electronegativity_2.00.1,RDF_electronegativity_2.01,RDF_electronegativity_2.03,RDF_electronegativity_2.04,...,RDF_vdWaalsVolume_28.53,RDF_vdWaalsVolume_29.01,RDF_vdWaalsVolume_29.51,RDF_vdWaalsVolume_30.00,PLD,LCD,density,VF,SAv,SAg
0,0,GIQXUA_clean,1.918899,0.017520,2.747258,0.000605,0.000606,0.000607,0.000609,0.000614,...,0.0,0.0,0.0,0.0,3.86870,4.89334,1.792830,0.13844,2675.750,1492.470
1,1,JALPAQ_clean,0.050226,0.027788,0.008087,0.000861,0.000870,0.000888,0.000916,0.000954,...,0.0,0.0,0.0,0.0,2.90553,3.97550,1.660880,0.01992,841.869,506.881
2,2,SAQQEH_clean,2.006302,0.249367,0.533491,0.000771,0.000781,0.000802,0.000833,0.000876,...,0.0,0.0,0.0,0.0,9.13279,10.56525,0.914389,0.35942,2030.210,2220.290
3,3,BEFNEH_clean,1.036156,0.033717,1.343768,0.000899,0.000899,0.000900,0.000903,0.000910,...,0.0,0.0,0.0,0.0,3.33494,4.26940,2.165640,0.07290,1864.990,861.172
4,4,GELVID_clean,0.552087,0.064768,0.027879,0.000820,0.000831,0.000852,0.000884,0.000928,...,0.0,0.0,0.0,0.0,2.43534,3.61901,1.574130,0.02776,1334.150,847.547


In [3]:
'''
  - read data into dataframe
  - generate class data
  - extract and split feature and class data
  - scale feature data
'''


df = df.sample(frac=1).reset_index(drop=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
# df = df[df.framework.str.startswith('str_')]


train_df_full, test_df = train_test_split(df, test_size=0.2, random_state=2020)

train_df, val_df = train_test_split(train_df_full, test_size=0.1, random_state=2020)

X_train = train_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_train_CO2 = train_df.iloc[:, 4:].to_numpy().astype(np.float32)

y_CO2 = train_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2 = train_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2 = train_df.SO2_tp_lod.to_numpy().astype(np.float32)

X_val = val_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_val_CO2 = val_df.iloc[:, 4:].to_numpy().astype(np.float32)
y_CO2_val = val_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2_val = val_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2_val = val_df.SO2_tp_lod.to_numpy().astype(np.float32)

X_test = test_df.iloc[:, 4:].to_numpy().astype(np.float32)
X_test_CO2 = test_df.iloc[:, 4:].to_numpy().astype(np.float32)
y_CO2_test =  test_df.CO2_tp_lod.to_numpy().astype(np.float32)
y_N2_test = test_df.N2_tp_lod.to_numpy().astype(np.float32)
y_SO2_test = test_df.SO2_tp_lod.to_numpy().astype(np.float32)


# y_CO2[100:1000] = -1

sc = StandardScaler()


X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [4]:
df.head()

,framework,CO2_tp_lod,N2_tp_lod,SO2_tp_lod,RDF_electronegativity_2.00,RDF_electronegativity_2.00.1,RDF_electronegativity_2.01,RDF_electronegativity_2.03,RDF_electronegativity_2.04,RDF_electronegativity_2.07,...,RDF_vdWaalsVolume_28.53,RDF_vdWaalsVolume_29.01,RDF_vdWaalsVolume_29.51,RDF_vdWaalsVolume_30.00,PLD,LCD,density,VF,SAv,SAg
0,str_m3_o6_o22_f0_nbo_sym_88_faps_mpnn_charges,0.220230,0.240675,0.023841,0.000736,0.000746,0.000766,0.000796,0.000837,0.000889,...,0.0,0.0,0.0,0.0,12.85588,16.84391,0.292539,0.67916,1633.04,5582.300
1,str_m29_o92_o101_f0_pts_sym_53_faps_mpnn_charges,0.615981,0.258884,2.902255,0.000699,0.000712,0.000737,0.000778,0.000834,0.000908,...,0.0,0.0,0.0,0.0,9.54271,11.16429,0.609476,0.42774,1950.97,3201.070
2,GESNOI_clean,3.262913,0.010957,1.645783,0.000920,0.000924,0.000933,0.000947,0.000967,0.000993,...,0.0,0.0,0.0,0.0,3.48820,6.06741,1.684730,0.06924,1427.64,847.399
3,str_m3_o153_o155_f0_fsc_sym_75_faps_mpnn_charges,1.609211,0.439934,0.874886,0.000717,0.000730,0.000757,0.000799,0.000857,0.000934,...,0.0,0.0,0.0,0.0,5.20687,8.06086,0.756559,0.36200,2489.46,3290.510
4,str_m3_o153_o164_f0_fsc_sym_1_faps_mpnn_charges,0.628296,0.232043,0.153936,0.000693,0.000705,0.000728,0.000765,0.000815,0.000881,...,0.0,0.0,0.0,0.0,7.37181,8.00676,0.791728,0.30210,2230.90,2817.770


In [5]:
input_ = tf.keras.Input(shape=(X_train.shape[1]))
x = tf.keras.layers.Dense(1200, activation='relu')(input_)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(400, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.Model(inputs=input_, outputs=x)

Metal device set to: Apple M1


2023-02-21 18:04:14.160440: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 18:04:14.161136: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
input_ = tf.keras.Input(shape=(X_train_CO2.shape[1]))
x = tf.keras.layers.Dense(1200, activation='relu')(input_)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(400, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(500, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

modelco2 = tf.keras.Model(inputs=input_, outputs=x)

In [7]:
cosy_model = CosyNet(
    model_config=[model.get_config(), modelco2.get_config(), model.get_config()],
    scalar=[1.0, 1.0, 1.0],
    max_layer_cutoff=-1,
    min_layer_cutoff=1,
    loss_fn=wasserstein_distance,
)

In [8]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00025)

losses = {
    'output_1':MaskedMeanSquaredError(),
    'output_2': MaskedMeanSquaredError(),
    'output_3': MaskedMeanSquaredError(),
    }

R2 = tfa.metrics.RSquare()

cosy_model.compile(
            optimizer=opt,
            loss=losses,
            metrics=[R2],
            )

In [9]:
cosy_model.fit(
    x=X_train, y=[y_N2, y_CO2, y_SO2],
    validation_data=(X_val, [y_N2_val, y_CO2_val, y_SO2_val]),
    batch_size=32, epochs = 4, verbose=1
)

Epoch 1/4


2023-02-21 18:04:14.664560: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-21 18:04:14.833353: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:04:14.835557: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:04:14.869584: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:04:14.871894: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:04:14.910068: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:04:14.912980: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INT

878/878 [==============================] - ETA: 0s - loss: 2.2189 - output_1_loss: 0.0143 - output_2_loss: 0.2375 - output_3_loss: 0.6930 - output_1_r_square: -0.9087 - output_2_r_square: 0.3387 - output_3_r_square: 0.4768 - scaled_soft_loss: 0.4247

2023-02-21 18:06:39.603014: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:06:39.605323: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:06:39.640090: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:06:39.641995: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:06:39.911545: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:06:39.913624: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at functional_ops.cc:374 : INTERNAL: No function library
2023-02-21 18:06:39.944964: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed 

878/878 [==============================] - 155s 173ms/step - loss: 2.2189 - output_1_loss: 0.0143 - output_2_loss: 0.2375 - output_3_loss: 0.6930 - output_1_r_square: -0.9087 - output_2_r_square: 0.3387 - output_3_r_square: 0.4768 - scaled_soft_loss: 0.4247 - val_loss: 2.0496 - val_output_1_loss: 0.0050 - val_output_2_loss: 0.1726 - val_output_3_loss: 0.5605 - val_output_1_r_square: 0.3028 - val_output_2_r_square: 0.4894 - val_output_3_r_square: 0.5841 - val_scaled_soft_loss: 0.4372
Epoch 2/4
878/878 [==============================] - 173s 197ms/step - loss: 2.0810 - output_1_loss: 0.0051 - output_2_loss: 0.1814 - output_3_loss: 0.5636 - output_1_r_square: 0.3196 - output_2_r_square: 0.4950 - output_3_r_square: 0.5745 - scaled_soft_loss: 0.4436 - val_loss: 1.9957 - val_output_1_loss: 0.0021 - val_output_2_loss: 0.1500 - val_output_3_loss: 0.5053 - val_output_1_r_square: 0.7107 - val_output_2_r_square: 0.5564 - val_output_3_r_square: 0.6251 - val_scaled_soft_loss: 0.4461
Epoch 3/4
878/8

KeyboardInterrupt: 

In [ ]:
models = cosy_model.get_models()

In [ ]:
params = cosy_model._get_parameters()

In [ ]:
from cosy.losses import trace_norm, squared_frobenius_norm

In [ ]:
trace_norm(params, [0.5, 0.5, 0.5])

In [ ]:
squared_frobenius_norm(params, [0.5, 0.5, 0.5])

In [ ]:
from sklearn.metrics import r2_score
cosy_model.evaluate(X_test, [y_N2_test, y_CO2_test, y_SO2_test])
predictions = cosy_model.predict(X_test)

R2 = tfa.metrics.RSquare()

pred1 = np.array(predictions[0]).reshape(-1)
pred2 = np.array(predictions[1]).reshape(-1)
pred3 = np.array(predictions[2]).reshape(-1)


print(r2_score(y_N2_test, pred1))
print(r2_score(y_CO2_test, pred2))
print(r2_score(y_SO2_test, pred3))

In [ ]:
import itertools


def squared_frobenius_norm(parameters, lambdas):
    """Computes the overall loss for soft parameter sharing between multiple neural networks."""
    losses = []
    for W, lambd in zip(parameters, lambdas):
        losses.append(pairwise_loss_squared_frobenius(W, lambd))
    return tf.reduce_sum(losses)

In [ ]:
import numpy as np
parameters = np.array([
        [tf.zeros((2, 2)), tf.ones((2, 2)), tf.zeros((2, 2))],
        [tf.ones((2, 2)), tf.ones((2, 2)), tf.ones((2, 2))],
        [tf.ones((2, 2)), tf.zeros((2, 2)), tf.ones((2, 2))],
    ])

lambdas = [0.5, 0.5, 0.5]

In [ ]:
loss = 0
for lam, param in zip(lambdas, parameters):
    norm1 = np.linalg.norm(param[0] - param[1], ord='fro') ** 2
    norm2 = np.linalg.norm(param[0] - param[2], ord='fro') ** 2
    norm3 = np.linalg.norm(param[1] - param[2], ord='fro') ** 2
    loss += lam * (norm1 + norm2 + norm3)

In [ ]:
loss

In [ ]:
loss = 0
for lam, param in zip(lambdas, parameters):
    norm1 = np.linalg.norm(param[0] - param[1], ord='nuc') ** 2
    norm2 = np.linalg.norm(param[0] - param[2], ord='nuc') ** 2
    norm3 = np.linalg.norm(param[1] - param[2], ord='nuc') ** 2
    loss += lam * (norm1 + norm2 + norm3)

In [ ]:
loss

In [ ]:
weights = [np.array([[1, 2], [3, 4]]), np.array([[2, 4], [6, 8]])]

In [ ]:
def pairwise_loss_squared_frobenius(W, lambd):
    """Computes the pairwise weight-sharing loss between a list of weight matrices."""
    squared_norms = []
    for Wi, Wj in itertools.combinations(W, 2):
        squared_norms.append(tf.norm(Wi - Wj, ord="fro", axis=(0, 1)) ** 2)
    loss = lambd * tf.reduce_sum(squared_norms)
    return loss


In [ ]:
norm1 = np.linalg.norm(weights[0] - weights[1], ord='fro') ** 2

In [ ]:
norm1

In [ ]:
W_tf = [tf.constant(Wi, dtype=tf.float32) for Wi in weights]
pairwise_loss_squared_frobenius(W_tf, 1.0)